In [2]:
import os
import sys
import json
import pandas as pd
from datetime import datetime

# Add project root (the directory that contains "src")
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# --- Imports from your project ---
from src.pipelines.FewShotPipeline import FewShotPipeline
from src.app.Evaluator import Evaluator
from src.llms.LLM_Wrappers import AbstractLLM

examples = [1, 2, 3, 4, 5]
llm = AbstractLLM.from_name("gpt-4o-mini")

pipeline = FewShotPipeline(
    llm=llm,
    input_path="../src/data/Q17_Annotated_Responses.json",
    example_ids=examples,
    output_dir="outputs/",
    output_name="FewShot_gpt-4o-mini_mult",
    use_cache=False
)

# 3. Run on a specific entry (e.g., ID 3)
# This will load the data, look at ID 1 and 2 for context, annotate ID 3, and save.
result = pipeline.run_multiple([12, 13, 14])

Annotating entries: 100%|███████████████████| 3/3 [00:11<00:00,  3.87s/entry, Last: 4.73s]

✅ Annotated batch of 3 entries saved (partial file) to outputs/Q17_Annotated_Responses_FewShot_gpt-4o-mini_mult_annotated.json


# K-Fold Cross Validation

## 10-Fold Validation (90/10 train/test split)

In [4]:
import os
import sys
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from datetime import datetime

# --- Paths Setup ---
# Add project root to sys.path if not present
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# --- Imports ---
from src.pipelines.FewShotPipeline import FewShotPipeline
from src.app.Evaluator import Evaluator
from src.llms.LLM_Wrappers import AbstractLLM

# --- Configuration ---
INPUT_FILE_PATH = "../src/data/Q17_Annotated_Responses.json"
OUTPUT_DIR = "outputs/experiments/"
MODEL_NAME = "gpt-4o-mini"
NUM_EXPERIMENTS = 3
K_FOLDS = 10

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Load Data & Extract IDs ---
with open(INPUT_FILE_PATH, "r") as f:
    data = json.load(f)

# Extract all available IDs from the dataset
all_ids = np.array([entry['id'] for entry in data['answers']])
print(f"Loaded {len(all_ids)} entries from {os.path.basename(INPUT_FILE_PATH)}")

# --- Initialize LLM ---
llm = AbstractLLM.from_name(MODEL_NAME)

# --- Storage for Results ---
all_results = []

print(f"Starting {NUM_EXPERIMENTS} Experiments with {K_FOLDS}-Fold Cross Validation...")
print("-" * 60)

for exp_num in range(1, NUM_EXPERIMENTS + 1):
    print(f"\n🚀 STARTING EXPERIMENT {exp_num}/{NUM_EXPERIMENTS}")
    
    # Initialize K-Fold with shuffling
    # random_state ensures different splits for each experiment
    kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42 + exp_num)
    
    for fold_num, (train_index, test_index) in enumerate(kf.split(all_ids), 1):
        
        # 1. Split IDs
        # train_ids (90%) -> used as Examples
        # test_ids (10%)  -> used as Targets to annotate
        train_ids = all_ids[train_index].tolist()
        test_ids = all_ids[test_index].tolist()
        
        # 2. Construct Unique Output Name
        # Format: Exp_X_Fold_Y_Model_Filename
        base_name = os.path.splitext(os.path.basename(INPUT_FILE_PATH))[0]
        unique_name = f"Exp{exp_num}_Fold{fold_num}_{MODEL_NAME}_{base_name}"
        
        print(f"  > Processing Fold {fold_num}/{K_FOLDS} (Targets: {len(test_ids)} entries)...")

        # 3. Initialize Pipeline with current Train IDs as Examples
        pipeline = FewShotPipeline(
            llm=llm,
            input_path=INPUT_FILE_PATH,
            example_ids=train_ids,
            output_dir=OUTPUT_DIR,
            output_name=unique_name,
            use_cache=False # Disable cache to ensure fresh runs for experiments
        )
        
        # 4. Run Pipeline on Test IDs
        # This saves the partial file containing examples + annotated targets
        output_path = pipeline.run_multiple(test_ids) # This returns a list, but pipeline saves file internally
        
        # The pipeline class saves to self.output_path. We reconstruct that path or rely on return
        # Since run_multiple returns entries, we can reconstruct path or use the attribute if accessible.
        # However, run_multiple writes to disk. Let's rely on the predictable path structure from pipeline:
        actual_output_path = pipeline.output_path
        
        # 5. Evaluate
        # Compare the Auto-Annotated File against the original Ground Truth File
        evaluator = Evaluator(auto_path=actual_output_path, gt_path=INPUT_FILE_PATH)
        
        # Get metrics (min_confidence can be adjusted)
        metrics = evaluator.evaluate_precision_recall(min_confidence=0.5)
        
        # 6. Store Metrics
        global_metrics = metrics['global']
        result_row = {
            "Experiment": exp_num,
            "Fold": fold_num,
            "Model": MODEL_NAME,
            "Train_Size": len(train_ids),
            "Test_Size": len(test_ids),
            "Precision": global_metrics['precision'],
            "Recall": global_metrics['recall'],
            "F1_Score": global_metrics['f1-score'],
            "Evaluated_Entries": global_metrics['evaluated_entries']
        }
        all_results.append(result_row)
        
        # Optional: Print Fold Result
        print(f"    -> Fold {fold_num} Result: F1={global_metrics['f1-score']:.3f} "
              f"(P={global_metrics['precision']:.3f}, R={global_metrics['recall']:.3f})")

# --- Final Aggregation & Saving ---

# Create DataFrame
df_results = pd.DataFrame(all_results)

# 1. Calculate Average per Experiment
exp_summary = df_results.groupby("Experiment")[["Precision", "Recall", "F1_Score"]].mean()

# 2. Calculate Grand Average (Average of all folds across all experiments)
grand_average = df_results[["Precision", "Recall", "F1_Score"]].mean()

# 3. Save to CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = f"Experiment_Results_{MODEL_NAME}_{timestamp}.csv"
csv_path = os.path.join(OUTPUT_DIR, csv_filename)
df_results.to_csv(csv_path, index=False)

print("\n" + "="*60)
print(f"🏁 EXPERIMENT COMPLETE")
print("="*60)
print(f"\n--- Summary by Experiment (Average of {K_FOLDS} Folds) ---")
print(exp_summary)

print(f"\n--- Grand Average (Across all {NUM_EXPERIMENTS * K_FOLDS} runs) ---")
print(grand_average)

print(f"\n📄 Detailed results saved to: {csv_path}")

Loaded 58 entries from Q17_Annotated_Responses.json
Starting 3 Experiments with 10-Fold Cross Validation...
------------------------------------------------------------

🚀 STARTING EXPERIMENT 1/3
  > Processing Fold 1/10 (Targets: 6 entries)...


Annotating entries: 100%|███████████████████| 6/6 [00:12<00:00,  2.00s/entry, Last: 1.77s]


✅ Annotated batch of 6 entries saved (partial file) to outputs/experiments/Q17_Annotated_Responses_Exp1_Fold1_gpt-4o-mini_Q17_Annotated_Responses_annotated.json
✅ Aligned 6 common entries for evaluation. (Skipped 0 auto-entries not in GT).
    -> Fold 1 Result: F1=0.737 (P=0.636, R=0.875)
  > Processing Fold 2/10 (Targets: 6 entries)...


Annotating entries: 100%|███████████████████| 6/6 [00:08<00:00,  1.41s/entry, Last: 1.22s]


✅ Annotated batch of 6 entries saved (partial file) to outputs/experiments/Q17_Annotated_Responses_Exp1_Fold2_gpt-4o-mini_Q17_Annotated_Responses_annotated.json
✅ Aligned 6 common entries for evaluation. (Skipped 0 auto-entries not in GT).
    -> Fold 2 Result: F1=0.800 (P=0.857, R=0.750)
  > Processing Fold 3/10 (Targets: 6 entries)...


...

✅ Annotated batch of 6 entries saved (partial file) to outputs/experiments/Q17_Annotated_Responses_Exp3_Fold8_gpt-4o-mini_Q17_Annotated_Responses_annotated.json
✅ Aligned 6 common entries for evaluation. (Skipped 0 auto-entries not in GT).
    -> Fold 8 Result: F1=0.741 (P=0.714, R=0.769)
  > Processing Fold 9/10 (Targets: 5 entries)...


Annotating entries: 100%|███████████████████| 5/5 [00:11<00:00,  2.30s/entry, Last: 1.40s]


✅ Annotated batch of 5 entries saved (partial file) to outputs/experiments/Q17_Annotated_Responses_Exp3_Fold9_gpt-4o-mini_Q17_Annotated_Responses_annotated.json
✅ Aligned 5 common entries for evaluation. (Skipped 0 auto-entries not in GT).
    -> Fold 9 Result: F1=0.700 (P=0.636, R=0.778)
  > Processing Fold 10/10 (Targets: 5 entries)...


Annotating entries: 100%|███████████████████| 5/5 [00:11<00:00,  2.24s/entry, Last: 3.18s]

✅ Annotated batch of 5 entries saved (partial file) to outputs/experiments/Q17_Annotated_Responses_Exp3_Fold10_gpt-4o-mini_Q17_Annotated_Responses_annotated.json
✅ Aligned 5 common entries for evaluation. (Skipped 0 auto-entries not in GT).
    -> Fold 10 Result: F1=0.824 (P=0.778, R=0.875)

🏁 EXPERIMENT COMPLETE

--- Summary by Experiment (Average of 10 Folds) ---
            Precision    Recall  F1_Score
Experiment                               
1            0.665996  0.839394  0.731977
2            0.650032  0.796111  0.710850
3            0.665418  0.846368  0.742194

--- Grand Average (Across all 30 runs) ---
Precision    0.660482
Recall       0.827291
F1_Score     0.728341
dtype: float64

📄 Detailed results saved to: outputs/experiments/Experiment_Results_gpt-4o-mini_20251208_094150.csv


## 4-Fold Validation (75/25 train/test split)

In [5]:
import os
import sys
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from datetime import datetime

# --- Paths Setup ---
# Add project root to sys.path if not present
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# --- Imports ---
from src.pipelines.FewShotPipeline import FewShotPipeline
from src.app.Evaluator import Evaluator
from src.llms.LLM_Wrappers import AbstractLLM

# --- Configuration ---
INPUT_FILE_PATH = "../src/data/Q17_Annotated_Responses.json"
OUTPUT_DIR = "outputs/experiments/"
MODEL_NAME = "gpt-4o-mini"
NUM_EXPERIMENTS = 3
K_FOLDS = 4

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Load Data & Extract IDs ---
with open(INPUT_FILE_PATH, "r") as f:
    data = json.load(f)

# Extract all available IDs from the dataset
all_ids = np.array([entry['id'] for entry in data['answers']])
print(f"Loaded {len(all_ids)} entries from {os.path.basename(INPUT_FILE_PATH)}")

# --- Initialize LLM ---
llm = AbstractLLM.from_name(MODEL_NAME)

# --- Storage for Results ---
all_results = []

print(f"Starting {NUM_EXPERIMENTS} Experiments with {K_FOLDS}-Fold Cross Validation...")
print("-" * 60)

for exp_num in range(1, NUM_EXPERIMENTS + 1):
    print(f"\n🚀 STARTING EXPERIMENT {exp_num}/{NUM_EXPERIMENTS}")
    
    # Initialize K-Fold with shuffling
    # random_state ensures different splits for each experiment
    kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42 + exp_num)
    
    for fold_num, (train_index, test_index) in enumerate(kf.split(all_ids), 1):
        
        # 1. Split IDs
        # train_ids (90%) -> used as Examples
        # test_ids (10%)  -> used as Targets to annotate
        train_ids = all_ids[train_index].tolist()
        test_ids = all_ids[test_index].tolist()
        
        # 2. Construct Unique Output Name
        # Format: Exp_X_Fold_Y_Model_Filename
        base_name = os.path.splitext(os.path.basename(INPUT_FILE_PATH))[0]
        unique_name = f"Exp{exp_num}_Fold{fold_num}_{MODEL_NAME}_{base_name}"
        
        print(f"  > Processing Fold {fold_num}/{K_FOLDS} (Targets: {len(test_ids)} entries)...")

        # 3. Initialize Pipeline with current Train IDs as Examples
        pipeline = FewShotPipeline(
            llm=llm,
            input_path=INPUT_FILE_PATH,
            example_ids=train_ids,
            output_dir=OUTPUT_DIR,
            output_name=unique_name,
            use_cache=False # Disable cache to ensure fresh runs for experiments
        )
        
        # 4. Run Pipeline on Test IDs
        # This saves the partial file containing examples + annotated targets
        output_path = pipeline.run_multiple(test_ids) # This returns a list, but pipeline saves file internally
        
        # The pipeline class saves to self.output_path. We reconstruct that path or rely on return
        # Since run_multiple returns entries, we can reconstruct path or use the attribute if accessible.
        # However, run_multiple writes to disk. Let's rely on the predictable path structure from pipeline:
        actual_output_path = pipeline.output_path
        
        # 5. Evaluate
        # Compare the Auto-Annotated File against the original Ground Truth File
        evaluator = Evaluator(auto_path=actual_output_path, gt_path=INPUT_FILE_PATH)
        
        # Get metrics (min_confidence can be adjusted)
        metrics = evaluator.evaluate_precision_recall(min_confidence=0.5)
        
        # 6. Store Metrics
        global_metrics = metrics['global']
        result_row = {
            "Experiment": exp_num,
            "Fold": fold_num,
            "Model": MODEL_NAME,
            "Train_Size": len(train_ids),
            "Test_Size": len(test_ids),
            "Precision": global_metrics['precision'],
            "Recall": global_metrics['recall'],
            "F1_Score": global_metrics['f1-score'],
            "Evaluated_Entries": global_metrics['evaluated_entries']
        }
        all_results.append(result_row)
        
        # Optional: Print Fold Result
        print(f"    -> Fold {fold_num} Result: F1={global_metrics['f1-score']:.3f} "
              f"(P={global_metrics['precision']:.3f}, R={global_metrics['recall']:.3f})")

# --- Final Aggregation & Saving ---

# Create DataFrame
df_results = pd.DataFrame(all_results)

# 1. Calculate Average per Experiment
exp_summary = df_results.groupby("Experiment")[["Precision", "Recall", "F1_Score"]].mean()

# 2. Calculate Grand Average (Average of all folds across all experiments)
grand_average = df_results[["Precision", "Recall", "F1_Score"]].mean()

# 3. Save to CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = f"Experiment_Results_{MODEL_NAME}_{timestamp}.csv"
csv_path = os.path.join(OUTPUT_DIR, csv_filename)
df_results.to_csv(csv_path, index=False)

print("\n" + "="*60)
print(f"🏁 EXPERIMENT COMPLETE")
print("="*60)
print(f"\n--- Summary by Experiment (Average of {K_FOLDS} Folds) ---")
print(exp_summary)

print(f"\n--- Grand Average (Across all {NUM_EXPERIMENTS * K_FOLDS} runs) ---")
print(grand_average)

print(f"\n📄 Detailed results saved to: {csv_path}")

Loaded 58 entries from Q17_Annotated_Responses.json
Starting 3 Experiments with 4-Fold Cross Validation...
------------------------------------------------------------

🚀 STARTING EXPERIMENT 1/3
  > Processing Fold 1/4 (Targets: 15 entries)...


Annotating entries: 100%|█████████████████| 15/15 [00:24<00:00,  1.61s/entry, Last: 2.65s]


✅ Annotated batch of 15 entries saved (partial file) to outputs/experiments/Q17_Annotated_Responses_Exp1_Fold1_gpt-4o-mini_Q17_Annotated_Responses_annotated.json
✅ Aligned 15 common entries for evaluation. (Skipped 0 auto-entries not in GT).
    -> Fold 1 Result: F1=0.837 (P=0.783, R=0.900)
  > Processing Fold 2/4 (Targets: 15 entries)...


Annotating entries: 100%|█████████████████| 15/15 [00:30<00:00,  2.05s/entry, Last: 1.12s]


✅ Annotated batch of 15 entries saved (partial file) to outputs/experiments/Q17_Annotated_Responses_Exp1_Fold2_gpt-4o-mini_Q17_Annotated_Responses_annotated.json
✅ Aligned 15 common entries for evaluation. (Skipped 0 auto-entries not in GT).
    -> Fold 2 Result: F1=0.708 (P=0.654, R=0.773)
  > Processing Fold 3/4 (Targets: 14 entries)...


...

✅ Annotated batch of 15 entries saved (partial file) to outputs/experiments/Q17_Annotated_Responses_Exp3_Fold2_gpt-4o-mini_Q17_Annotated_Responses_annotated.json
✅ Aligned 15 common entries for evaluation. (Skipped 0 auto-entries not in GT).
    -> Fold 2 Result: F1=0.756 (P=0.654, R=0.895)
  > Processing Fold 3/4 (Targets: 14 entries)...


Annotating entries: 100%|█████████████████| 14/14 [00:24<00:00,  1.78s/entry, Last: 2.58s]


✅ Annotated batch of 14 entries saved (partial file) to outputs/experiments/Q17_Annotated_Responses_Exp3_Fold3_gpt-4o-mini_Q17_Annotated_Responses_annotated.json
✅ Aligned 14 common entries for evaluation. (Skipped 0 auto-entries not in GT).
    -> Fold 3 Result: F1=0.818 (P=0.750, R=0.900)
  > Processing Fold 4/4 (Targets: 14 entries)...


Annotating entries: 100%|█████████████████| 14/14 [00:27<00:00,  1.98s/entry, Last: 2.00s]

✅ Annotated batch of 14 entries saved (partial file) to outputs/experiments/Q17_Annotated_Responses_Exp3_Fold4_gpt-4o-mini_Q17_Annotated_Responses_annotated.json
✅ Aligned 14 common entries for evaluation. (Skipped 0 auto-entries not in GT).
    -> Fold 4 Result: F1=0.764 (P=0.724, R=0.808)

🏁 EXPERIMENT COMPLETE

--- Summary by Experiment (Average of 4 Folds) ---
            Precision    Recall  F1_Score
Experiment                               
1            0.644100  0.795574  0.710460
2            0.651786  0.781283  0.709337
3            0.670885  0.858941  0.751010

--- Grand Average (Across all 12 runs) ---
Precision    0.655590
Recall       0.811932
F1_Score     0.723602
dtype: float64

📄 Detailed results saved to: outputs/experiments/Experiment_Results_gpt-4o-mini_20251208_095314.csv
